In [1]:
!git clone https://github.com/nguyenduchuyiu/PCB-Diffusion-AD.git

Cloning into 'PCB-Diffusion-AD'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 58 (delta 26), reused 51 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 41.35 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [2]:
%cd PCB-Diffusion-AD

/kaggle/working/PCB-Diffusion-AD


In [3]:
!pip install imgaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 15.0 MB/s eta 0:00:00a 0:00:01


In [ ]:
%%writefile args/args1.json
{
  "img_size": [256,256],
  "Batch_Size": 4,
  "EPOCHS": 3000,
  "T": 1000,
  "base_channels": 128,
  "beta_schedule": "linear",
  "loss_type": "l2",
  "diffusion_lr": 1e-4,
  "seg_lr": 1e-5,
  "random_slice": true,
  "weight_decay": 0.0,
  "save_imgs":true,
  "save_vids":false, 
  "dropout":0,
  "attention_resolutions":"32,16,8",
  "num_heads":4,
  "num_head_channels":-1,
  "noise_fn":"gauss",
  "channels":3,
  "data_name":"RealIAD",
  "data_root_path":"/kaggle/input/pcb-dataset",
  "anomaly_source_path":"/kaggle/input/pcb-dataset/dtd",
  "noisier_t_range":600,
  "less_t_range":300,
  "condition_w":1,
  "eval_normal_t":200,
  "eval_noisier_t":400,
  "output_path":"outputs",
  "gradient_accumulation_steps": 4,
  "use_mixed_precision": true
}


Overwriting args/args1.json


In [5]:
import torch, gc

gc.collect() 
torch.cuda.empty_cache()  
torch.cuda.ipc_collect()  

from random import seed
import torch
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
from torch.utils.data import DataLoader
from models.Recon_subnetwork import UNetModel, update_ema_params
from models.Seg_subnetwork import SegmentationSubNetwork
from tqdm import tqdm
import torch.nn as nn
from data.dataset_beta_thresh import RealIADTrainDataset, RealIADTestDataset
from math import exp
import torch.nn.functional as F
from models.DDPM import GaussianDiffusionModel, get_beta_schedule
from scipy.ndimage import gaussian_filter
from skimage.measure import label, regionprops
from sklearn.metrics import roc_auc_score,auc,average_precision_score
import pandas as pd
from collections import defaultdict
from train import defaultdict_from_json, train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check for multiple GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")
if num_gpus > 1:
    print(f"Using {num_gpus} GPUs for training")
elif num_gpus == 1:
    print("Using single GPU for training")
else:
    print("Using CPU for training")

# read file from argument
file = "args1.json"
# load the json args
with open(f'./args/{file}', 'r') as f:
    args = json.load(f)
args['arg_num'] = file[4:-5]
args = defaultdict_from_json(args)


real_iad_classes = os.listdir(os.path.join(args["data_root_path"], args['data_name']))

for sub_class in real_iad_classes:   
    print("class", sub_class)
    
    subclass_path = os.path.join(args["data_root_path"], args['data_name'], sub_class)
    
    training_dataset = RealIADTrainDataset(
        subclass_path, sub_class, img_size=args["img_size"], args=args
    )
    testing_dataset = RealIADTestDataset(
        subclass_path, sub_class, img_size=args["img_size"]
    )
    class_type=args['data_name']
    

    print(file, args)     

    data_len = len(testing_dataset)
    
    # Calculate effective batch size considering multi-GPU and gradient accumulation
    base_batch_size = args['Batch_Size']
    gradient_accumulation_steps = args.get('gradient_accumulation_steps', 1)
    
    # For DataLoader, we use the base batch size
    dataloader_batch_size = base_batch_size
    
    # Total effective batch size
    total_effective_batch_size = dataloader_batch_size * gradient_accumulation_steps
    
    print(f"Batch size configuration:")
    print(f"  - Base batch size: {base_batch_size}")
    print(f"  - DataLoader batch size: {dataloader_batch_size} ({'Multi-GPU' if num_gpus > 1 else 'Single-GPU'})")
    print(f"  - Gradient accumulation steps: {gradient_accumulation_steps}")
    print(f"  - Total effective batch size: {total_effective_batch_size}")
    
    training_dataset_loader = DataLoader(training_dataset, batch_size=dataloader_batch_size,shuffle=True,num_workers=8,pin_memory=True,drop_last=True)
    test_loader = DataLoader(testing_dataset, batch_size=1,shuffle=False, num_workers=4)

    # make arg specific directories
    for i in [f'{args["output_path"]}/model/diff-params-ARGS={args["arg_num"]}/{sub_class}',
            f'{args["output_path"]}/diffusion-training-images/ARGS={args["arg_num"]}/{sub_class}',
             f'{args["output_path"]}/metrics/ARGS={args["arg_num"]}/{sub_class}']:
        try:
            os.makedirs(i)
        except OSError:
            pass


    train(training_dataset_loader, test_loader, args, data_len,sub_class,class_type,device, num_gpus)

Number of available GPUs: 2
Using 2 GPUs for training
class PCB5
args1.json defaultdict(<class 'str'>, {'img_size': [256, 256], 'Batch_Size': 4, 'EPOCHS': 1, 'T': 1000, 'base_channels': 128, 'beta_schedule': 'linear', 'loss_type': 'l2', 'diffusion_lr': 0.0001, 'seg_lr': 1e-05, 'random_slice': True, 'weight_decay': 0.0, 'save_imgs': True, 'save_vids': False, 'dropout': 0, 'attention_resolutions': '32,16,8', 'num_heads': 4, 'num_head_channels': -1, 'noise_fn': 'gauss', 'channels': 3, 'data_name': 'RealIAD', 'data_root_path': '/kaggle/input/pcb-dataset', 'anomaly_source_path': '/kaggle/input/pcb-dataset/dtd', 'noisier_t_range': 600, 'less_t_range': 300, 'condition_w': 1, 'eval_normal_t': 200, 'eval_noisier_t': 400, 'output_path': 'outputs', 'gradient_accumulation_steps': 4, 'use_mixed_precision': True, 'arg_num': '1'})
Batch size configuration:
  - Base batch size: 4
  - DataLoader batch size: 4 (Multi-GPU)
  - Gradient accumulation steps: 4
  - Total effective batch size: 16


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/kaggle/working/PCB-Diffusion-AD/train.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if use_mixed_precision else None
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Wrapping models with DataParallel for 2 GPUs
Model Parameters:
  - UNet Model: 131,654,403 parameters
  - Segmentation Model: 28,373,569 parameters
  - Total Trainable Parameters: 160,027,972 parameters
  - Memory (approx): 610.5 MB (FP32)
Mixed Precision (FP16): Enabled
Gradient Accumulation Steps: 4
Effective Batch Size: 16


Epoch:0, Train loss: 336.367: 100%|██████████| 493/493 [11:47<00:00,  1.43s/it]


In [ ]:
import torch, gc

gc.collect() 
torch.cuda.empty_cache()  
torch.cuda.ipc_collect()  
from eval import testing, load_parameters, defaultdict_from_json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check for multiple GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")
if num_gpus > 1:
    print(f"Using {num_gpus} GPUs for evaluation")
elif num_gpus == 1:
    print("Using single GPU for evaluation")
else:
    print("Using CPU for evaluation")
file = "args1.json"
# load the json args
with open(f'./args/{file}', 'r') as f:
    args = json.load(f)
args['arg_num'] = file[4:-5]
args = defaultdict_from_json(args)
real_iad_classes = os.listdir(os.path.join(args["data_root_path"], args['data_name']))

current_classes = real_iad_classes
checkpoint_types = ['best', 'last']

for sub_class in current_classes:
    for checkpoint_type in checkpoint_types:
        try:
            args, output = load_parameters(device, sub_class, checkpoint_type)
        except FileNotFoundError:
            print(f"Checkpoint {checkpoint_type} not found for class {sub_class}, skipping.")
            continue

        print(f"args{args['arg_num']}")
        print("class", sub_class)
        
        in_channels = args["channels"]

        unet_model = UNetModel(args['img_size'][0], args['base_channels'], channel_mults=args['channel_mults'], dropout=args[
                    "dropout"], n_heads=args["num_heads"], n_head_channels=args["num_head_channels"],
                in_channels=in_channels
                ).to(device)

        seg_model = SegmentationSubNetwork(in_channels=6, out_channels=1).to(device)

        # Load model states
        unet_model.load_state_dict(output["unet_model_state_dict"])
        unet_model.to(device)
        
        seg_model.load_state_dict(output["seg_model_state_dict"])
        seg_model.to(device)
        
        # Enable multi-GPU for evaluation if available
        if num_gpus > 1:
            print(f"Wrapping models with DataParallel for {num_gpus} GPUs")
            unet_model = torch.nn.DataParallel(unet_model)
            seg_model = torch.nn.DataParallel(seg_model)
        
        unet_model.eval()
        seg_model.eval()

        print("EPOCH:", output['n_epoch'])

        testing_dataset = RealIADTestDataset(
            args["data_root_path"], sub_class, img_size=args["img_size"]
        )
        class_type = args['data_name']
                
        data_len = len(testing_dataset) 
        test_loader = DataLoader(testing_dataset, batch_size=1, shuffle=False, num_workers=4)

        # make arg specific directories
        for i in [f'{args["output_path"]}/metrics/ARGS={args["arg_num"]}/{sub_class}']:
            try:
                os.makedirs(i)
            except OSError:
                pass

        testing(test_loader, args, unet_model, seg_model, data_len, sub_class, class_type, checkpoint_type, device)


Number of available GPUs: 2
Using 2 GPUs for evaluation
checkpoint outputs/model/diff-params-ARGS=1/PCB5/params-best.pt
Checkpoint best not found for class PCB5, skipping.
checkpoint outputs/model/diff-params-ARGS=1/PCB5/params-last.pt
args1
class PCB5
Wrapping models with DataParallel for 2 GPUs
EPOCH: 1
data_len 2782


 42%|████▏     | 1166/2782 [19:36<31:19,  1.16s/it]  